In [1]:
import pandas as pd
import altair as alt

In [2]:
results_df = pd.read_csv("../data/interim/results_table.csv")

In [3]:
results_df["Score: "] = pd.to_numeric(results_df["Score: "]) 
results_df = results_df.rename(columns={"Score: ": 'Score', 'Test Set:': 'Test Set', 'Race:':'Race', 
                                        "Model: ":"Model", "Metric:": "Metric"})
results_df["Metric"] = results_df["Metric"].apply(lambda x: "F1 Weighted Average" if str(x) == "F1-weighted-avg"else x)
results_df["Race"] = results_df["Race"].apply(lambda x: str(x).title() if str(x) != 'api' else str(x).upper())
baseline_results = results_df.loc[(results_df["Model"] == "LUKE-Baseline") & (results_df["Test Set"] != "CoNLL-03 Test")]
custom_results = results_df.loc[(results_df["Model"] == "LUKE-Custom") & (results_df["Test Set"] != "CoNLL-03 Test")]

In [5]:
def plot_breakdown(input_df, title):
    groups = ["Best Performing Race Group", "Other 3 Race Groups"]
    scores = [list(input_df.loc[input_df["Race"]=="White"]["Score"])[0], input_df.loc[input_df["Race"]!="White"]["Score"].mean()]
    
    df = pd.DataFrame(columns=["Group", "F1 Score"])
    df["Mean F1 Score"] = scores
    df["Group"] = groups
    
    chart = alt.Chart(df).mark_bar().encode(
    x=alt.X('Group:O', axis=alt.Axis(labelAngle=0, ticks=False), sort={"encoding":"y", "order":"descending"}),
    y=alt.Y('Mean F1 Score:Q', axis=alt.Axis(labelAngle=0))
    ).properties(width = 300, height = 300, title=title)

    return chart

In [6]:
plot_breakdown(baseline_results, "Baseline LUKE Results") | plot_breakdown(custom_results, "Custom LUKE Results")

alt.HConcatChart(...)

In [7]:
def plot_all_group_chart(input_df, title):
    sort_order = ["White", "Hispanic", "API", "Black"]
    chart = alt.Chart(input_df).mark_bar().encode(
        x=alt.X('Race:O', axis=alt.Axis(labelAngle=0, ticks=False), sort=sort_order),
        y=alt.Y('Score:Q', axis=alt.Axis(labelAngle=0), title="F1 Score")
    ).properties(width = 300, height = 300, title=title)
    return chart

In [8]:
baseline_chart = plot_all_group_chart(baseline_results, "Baseline LUKE Results")
custom_chart = plot_all_group_chart(custom_results, "Custom LUKE Results")

In [9]:
baseline_chart | custom_chart

alt.HConcatChart(...)

In [13]:
def plot_conll_results(input_df):
    conll_results = input_df.loc[(results_df["Test Set"] == "CoNLL-03 Test")]
    chart = alt.Chart(conll_results).mark_bar().encode(
    x=alt.X('Model:N', axis=alt.Axis(labelAngle=0, ticks=False)),
    y=alt.Y('Score:Q', axis=alt.Axis(labelAngle=0), title="F1 Score"), 
    color=alt.Color("Model:N"), 
    column=alt.Column("Metric:N", title="Metric")
).properties(width = 200, height = 300, title="CoNLL-2003 Test Results")
    return chart

In [14]:
conll_results = plot_conll_results(results_df)

In [15]:
conll_results

alt.Chart(...)